In [22]:
#def weighted_average_score(df, k=0.8):
    #n_views = df.groupby('movieId', sort=False).movieId.count()
    #ratings = df.groupby('movieId', sort=False).rating.mean()
    #scores = ((1-k)*(n_views/n_views.max()) + 
              #k*(ratings/ratings.max())).to_numpy().argsort()[::-1]
    #df_deduped = df.groupby('movieId', sort=False).agg({'title':'first', 
                                                         #'genres':'first', 
                                                         #'rating':'mean'})
    #return df_deduped.assign(views=n_views).iloc[scores]

In [24]:
#df = movies.merge(ratings)
#weighted_average_score(df).head(10)

In [51]:
#cuanta cuantas peliculas de cada genero hay
#genre_popularity = (movies.genres.str.split('|')
                      #.explode()
                      #.value_counts()
                      #.sort_values(ascending=False))
#genre_popularity.head(10)

In [43]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from itertools import combinations
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [27]:
tf = TfidfVectorizer(analyzer=lambda s: (c for i in range(1,4)
                                             for c in combinations(s.split('|'), r=i)))
tfidf_matrix = tf.fit_transform(movies['genres'])

In [30]:
cosine_sim = cosine_similarity(tfidf_matrix)

In [31]:
cosine_sim_df = pd.DataFrame(cosine_sim, index=movies['title'], columns=movies['title'])

#print('Shape:', cosine_sim_df.shape)
#cosine_sim_df.sample(10, axis=1).round(2)

In [44]:
ratings = pd.read_csv("ratings.csv", encoding='latin-1', usecols=['userId', 'movieId', 'rating'])
movies = pd.read_csv("movies.csv", encoding='latin-1' ,usecols=['movieId', 'title', 'genres'])

In [45]:
def genre_recommendations(i, M, items, k=10):
    """
    Recommends movies based on a similarity dataframe

    Parameters
    ----------
    i : str
        Movie (index of the similarity dataframe)
    M : pd.DataFrame
        Similarity dataframe, symmetric, with movies as indices and columns
    items : pd.DataFrame
        Contains both the title and some other features used to define similarity
    k : int
        Amount of recommendations to return

    """
    ix = M.loc[:,i].to_numpy().argpartition(range(-1,-k,-1))
    closest = M.columns[ix[-1:-(k+2):-1]]
    closest = closest.drop(i, errors='ignore')
    return pd.DataFrame(closest).merge(items).head(k)

In [46]:
movies[movies.title.eq('Toy Story (1995)')]

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy


In [49]:
genre_recommendations('Coco (2017)', cosine_sim_df, movies[['title', 'genres']])


,title,genres
0,Ice Age: A Mammoth Christmas (2011),Adventure|Animation|Children
1,Return to Never Land (2002),Adventure|Animation|Children
2,Digimon: The Movie (2000),Adventure|Animation|Children
3,Karlson Returns (1970),Adventure|Animation|Children
4,Balto (1995),Adventure|Animation|Children
5,Adventures of Mowgli: The Kidnapping (1968),Adventure|Animation|Children
6,Dinosaur (2000),Adventure|Animation|Children
7,Brother Bear (2003),Adventure|Animation|Children
8,101 Dalmatians (One Hundred and One Dalmatians...,Adventure|Animation|Children
9,Pocahontas II: Journey to a New World (1998),Adventure|Animation|Children
